<a href="https://colab.research.google.com/github/dmsxor/euntaek/blob/main/%EB%AF%B8%EB%9E%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# 로컬 파일 업로드
uploaded = files.upload()


Saving 마트.txt to 마트.txt


In [53]:
%%writefile Item.h
#ifndef ITEM_H
#define ITEM_H
#include <string>
//추상클래스
class Item {
public:
    virtual string getName() const = 0;
    virtual int getPrice() const = 0;
    virtual ~Item() {}
};
#endif // ITEM_H

Overwriting Item.h


In [49]:
%%writefile Product.cpp
#include "Item.h"
#include <string>

#include <iostream>

using namespace std;
// 구체 클래스
class Product : public Item {
private:
    string name;
    int price;

public:
    Product(const string& n, int p) : name(n), price(p) {}

    string getName() const override {
        return name;
    }

    int getPrice() const override {
        return price;
    }

    // 연산자 중복
    friend ostream& operator<<(ostream& os, const Product& product) {
        os << product.getName() << " - " << product.getPrice() << "원";
        return os;
    }
};


Overwriting Product.cpp


In [50]:
%%writefile Shop.cpp
#include "Item.h"
#include <vector>
#include <memory>

using namespace std;


// 제네릭 클래스
template <typename T>
class Shop {
private:
    vector<shared_ptr<T>> items;

public:
    void addItem(shared_ptr<T> item) {
        items.push_back(item);
    }

    // 함수 중복: 문자열과 가격으로 아이템 추가
    void addItem(const string& name, int price) {
        addItem(make_shared<Product>(name, price));
    }

    const vector<shared_ptr<T>>& getItems() const {
        return items;
    }
};

Overwriting Shop.cpp


In [35]:
%%writefile BudgetException.cpp
#include <stdexcept>
#include <string>

using namespace std;
// 예외 클래스
class BudgetException : public runtime_error {
public:
    BudgetException(const string& msg) : runtime_error(msg) {}
};

Overwriting BudgetException.cpp


In [54]:
%%writefile mirae.cpp
#include "BudgetException.cpp"
#include "Item.h"
#include "Product.cpp"
#include "Shop.cpp"
#include <iostream>
#include <vector>
#include <fstream>
#include <stdexcept>
#include <string>
#include <algorithm>
#include <memory>
#include <sstream>


using namespace std;

// 배낭 문제 알고리즘
void knapsack(int budget, const vector<shared_ptr<Item>>& items, const vector<bool>& essentials) {
    vector<shared_ptr<Item>> purchasedItems;
    int totalEssentialCost = 0;

    // 필수 물건의 총 가격 계산
    for (size_t i = 0; i < items.size(); ++i) {
        if (essentials[i]) {
            purchasedItems.push_back(items[i]);
            totalEssentialCost += items[i]->getPrice();
        }
    }

    if (totalEssentialCost > budget) {
        throw BudgetException("예산이 부족하여 물건을 구매할 수 없습니다.");
    }

    int remainingBudget = budget - totalEssentialCost;

    // 선택 물건 가격 기준 정렬
    vector<shared_ptr<Item>> optionalItems;
    for (size_t i = 0; i < items.size(); ++i) {
        if (!essentials[i]) {
            optionalItems.push_back(items[i]);
        }
    }

    sort(optionalItems.begin(), optionalItems.end(), [](const shared_ptr<Item>& a, const shared_ptr<Item>& b) {
        return a->getPrice() < b->getPrice();
    });

    // 선택 물건 구매
    for (const auto& item : optionalItems) {
        if (item->getPrice() <= remainingBudget) {
            purchasedItems.push_back(item);
            remainingBudget -= item->getPrice();
        }
    }

    // 결과 출력
    cout << "구매 목록:" << endl;
    for (const auto& item : purchasedItems) {
        cout << *dynamic_cast<Product*>(item.get()) << endl; // 연산자 중복 사용
    }
    cout << "남은 예산: " << remainingBudget << "원" << endl;

    // 파일 출력
    ofstream outFile("구매목록.txt");
    if (outFile.is_open()) {
        outFile << "구매 목록:" << endl;
        for (const auto& item : purchasedItems) {
            outFile << *dynamic_cast<Product*>(item.get()) << endl; // 연산자 중복 사용
        }
        outFile << "남은 예산: " << remainingBudget << "원" << endl;
        outFile.close();
    } else {
        throw runtime_error("구매 목록 파일을 열 수 없습니다.");
    }
}

// 파일에서 물건 읽기
void readItemsFromFile(const string& filename, Shop<Item>& shop) {
    ifstream file(filename);
    if (!file.is_open()) {
        throw runtime_error("파일을 열 수 없습니다.");
    }

    string name;
    int price;
    while (file >> name >> price) {
        shop.addItem(make_shared<Product>(name, price));
    }
}

int main() {
    Shop<Item> shop;

    // 물건 목록 읽기
    try {
        readItemsFromFile("마트.txt", shop);

        int itemCount = shop.getItems().size();
        cout << "물건 목록:" << endl;
        for (int i = 0; i < itemCount; ++i) {
            cout << i + 1 << ": " << shop.getItems()[i]->getName() << " - " << shop.getItems()[i]->getPrice() << "원" << endl;
        }

        int budget;
        cout << "예산을 입력하세요: ";
        cin >> budget;

        // 예산 입력 시 유효성 검사 
        if (cin.fail()) { // 숫자가 아닌 경우
            cerr << "예산은 숫자로 입력해야 합니다." << endl; // 오류 메시지 출력
            return 0; // 프로그램 종료
        }

        // 모든 물건의 가격 합계 계산
        int totalCost = 0;
        for (const auto& item : shop.getItems()) {
            totalCost += item->getPrice();
        }

        // 예산이 모든 물건의 가격보다 크거나 같은 경우
        if (budget >= totalCost) {
            cout << "모든 물건을 구매할 수 있습니다." << endl;
            return 0;
        }

        vector<bool> essentials(itemCount, false);
        cout << "필수 물건 번호를 입력하세요 (공백으로 구분): ";
        cin.ignore(); // 이전 입력에서 남은 개행 문자 무시
        string input;
        getline(cin, input);
        stringstream ss(input);
        int number;

        bool invalidInput = false; // 유효하지 않은 입력 플래그

        while (ss >> number) {
            if (number > 0 && number <= itemCount) {
                essentials[number - 1] = true; // 1-based index to 0-based index
            } else {
                invalidInput = true; // 유효하지 않은 번호가 입력되었음을 기록
            }
        }

        // 필수 물건 번호 입력 시 유효성 검사
        if (ss.fail() && !ss.eof()) { // 숫자가 아닌 경우
            cerr << "필수 물건 번호는 숫자로 입력해야 합니다." << endl; // 오류 메시지 출력
            return 0; // 프로그램 종료
        }

        if (invalidInput) {
            cout << "존재하지 않는 물건입니다." << endl;
            return 0; // 프로그램 종료
        }

        knapsack(budget, shop.getItems(), essentials);
    } catch (const BudgetException& e) {
        cerr << e.what() << endl;
    } catch (const runtime_error& e) {
        cerr << e.what() << endl;
    }

    return 0;
}


Overwriting mirae.cpp


In [56]:
!g++ mirae.cpp -o mirae
!./mirae

물건 목록:
1: 사과 - 2000원
2: 딸기 - 1500원
3: 포도 - 2500원
4: 바나나 - 3000원
5: 복숭아 - 3500원
6: 당근 - 1500원
7: 오이 - 1000원
8: 버섯 - 2000원
9: 양파 - 1200원
10: 두부 - 1700원
11: 칫솔 - 2300원
12: 치약 - 1800원
13: 샴푸 - 2200원
14: 린스 - 1400원
15: 바디워시 - 3300원
16: 새우깡 - 1600원
17: 홈런볼 - 1800원
18: 자갈치 - 1500원
19: 포카칩 - 1700원
예산을 입력하세요: 12345678
모든 물건을 구매할 수 있습니다.
